###  Implementation of a Low Pass FIR Filter with DMAs in SDSoC Ported to python

In [1]:
from pynq import Overlay,PL,Xlnk
from fir import general_const
import timeit

#load bitstream
libfile = general_const.LIBRARY
bitfile = general_const.BITFILE
fir_overlay = Overlay(bitfile)     
if PL.bitfile_name != bitfile:
    fir_overlay.download()
    
xlnk = Xlnk()
def mem_init(buflen):
    """ Allocate contiguous memory buffer
    """
    buf = xlnk.cma_alloc(buflen)
    return xlnk.cma_cast(buf, "int")

number=2000
n = dlen = 1000
din = mem_init(n*4)
win = mem_init(85)
dout = mem_init(n*4)

### ABI

In [2]:
import cffi

ffi0 = cffi.FFI()
ffi0.cdef("void _p0_cpp_FIR_1_noasync(int *x, int w[85], int *ret, "
                  "int datalen);")
lib0 = ffi0.dlopen(libfile)

# call accelerator
def abiresp():
    lib0._p0_cpp_FIR_1_noasync(din,win,dout,dlen)

print("Running the benchmarks")
print("Time taken by abi", number,"times",timeit.timeit(abiresp,number=number))

Running the benchmarks
Time taken by abi 2000 times 1.920367929036729


### API

In [3]:
# Steps taken for API mode:
#   1. run example_build.py
#   2. add "from _example import fir, lib" to __init__.py

from fir import ffi, lib

# call accelerator
def apiresp():
    lib._p0_cpp_FIR_1_noasync(din,win,dout,dlen)

print("Running the benchmarks")
print("Time taken by api", number,"times",timeit.timeit(apiresp,number=number))

Running the benchmarks
Time taken by api 2000 times 1.9022526310291141


### Init

In [4]:
number=1

def abiInit():
    ffi0 = cffi.FFI()
    ffi0.cdef("void _p0_cpp_FIR_1_noasync(int *x, int w[85], int *ret, "
                  "int datalen);")
    lib0 = ffi0.dlopen(libfile)

def apiInit():
    from fir import ffi, lib
    
print("Time taken by abi", number,"times",timeit.timeit(abiInit,number=number))
print("Time taken by api", number,"times",timeit.timeit(apiInit,number=number))

Time taken by abi 1 times 0.01767707703402266
Time taken by api 1 times 4.811998223885894e-05
